<a href="https://colab.research.google.com/github/mltngpot/Describer/blob/main/Project3_HTMLParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests
from bs4 import BeautifulSoup

#url = "https://www.purdue.edu/newsroom/purduetoday/2023/Q4/employees-can-now-connect-with-their-purdue-retirement-program-on-fidelity-netbenefits/"
#url = "https://www.purdue.edu/research/features/stories/new-ranking-demonstrates-purdue-prominence-in-ai-research-supercomputing/"
url = "https://www.purdue.edu/newsroom/purduetoday/2023/Q4/stay-safe-and-injury-free-this-winter-top-tips-for-preventing-falls-and-walking-safely/"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title using its ID
    title = soup.find('h1', id='main-heading')
    article_title = title.text.strip() if title else "Title not found"

    # Find the specific div with class "post-content__body"
    content_div = soup.find('div', class_='post-content__body')
    if content_div:
      # Extract text content from paragraphs and list items within this div
      paragraphs = [p.text.strip() for p in content_div.find_all(['p', 'li'])]

      # Extract image URLs within this div
      images = [img['src'] for img in content_div.find_all('img') if 'src' in img.attrs]

      # Combine all text
      article_text = "\n".join(paragraphs)
    else:
      print("The specified div with class 'post-content__body' was not found.")


    # Find the specific div with class "post-content__thumbnail" for images
    thumbnail_div = soup.find('div', class_='post-content__thumbnail')
    image_urls = []
    if thumbnail_div:
      # Extract image URLs
      image_urls = [img['src'] for img in thumbnail_div.find_all('img') if 'src' in img.attrs]
    else:
      print("The specified div with class 'post-content__thumbnail' was not found.")

    print("Title:")
    print(article_title)
    print("Article Text:")
    print(article_text)
    print("\nImages:")
    print(images)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



Title:
Stay safe and injury-free this winter: Top tips for preventing falls and walking safely
Article Text:
As the Indiana winter settles in, Environmental Health and Safety (EHS) is once again sharing safety and walking tips for navigating snowy and icy conditions. To avoid falls and injuries, EHS encourages the campus community to take the following precautions before heading out this season:
Plan ahead for routes and only take cleared and treated surfaces. Use pedestrian subwalks and skywalks to walk to buildings when possible.
Wear appropriate footwear. If wearing ice cleats, be sure to remove them indoors to avoid slipping.
Slow down and take shorter steps to help with traction.
Avoid carrying heavy or awkward loads.
Use handrails when using stairs or ramps.
Consider using an elevator, if available, as indoor stairs may also be wet due to tracked-in snow and ice. Falls on stairs can lead to severe injuries, especially when slippery conditions are present.
Use a door or seat to pr